In [7]:
import os
import time
import json
import pandas as pd
import requests
from dotenv import load_dotenv

# Load .env and fetch API key
load_dotenv()
api_key = os.getenv("riot_api_key")

# --- DICTIONARIES (Replace with your own JSON loads if needed) ---

# keys to go from numbers to actual names
perk = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json'
perk_styles = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json'
champ = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-summary.json'
item = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json'

perk_json = requests.get(perk).json()
perk_styles_json = requests.get(perk_styles).json()
champ_json = requests.get(champ).json()
item_json = requests.get(item).json()

def json_extract(obj, key):
    arr = []
    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict, list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr
    return extract(obj, arr, key)

perk_ids = json_extract(perk_json, 'id')
perk_names = json_extract(perk_json, 'name')
perk_dict = dict(map(lambda i, j : (int(i), j), perk_ids, perk_names))

perk_styles_ids = json_extract(perk_styles_json, 'id')
perk_styles_names = json_extract(perk_styles_json, 'name')
perk_styles_dict = dict(map(lambda i, j : (int(i), j), perk_styles_ids, perk_styles_names))

champ_dict = {int(champ['id']): champ['name'] for champ in champ_json}
item_dict = {int(item['id']): item['name'] for item in item_json}

# --- CONFIG ---
pd.options.display.max_columns = 150
region = 'americas'
platform = 'na1'
name_cache, rank_cache, summoner_id_cache = {}, {}, {}

# --- INPUT ---
game_name = input("🎮 Enter gameName: ").strip()
tag_line = input("🏷️ Enter tagLine (e.g., NA1): ").strip()
games_per_player = int(input("🕹️ Games per player: "))
max_total_games = int(input("📊 Max total games: "))

# --- Rate limit ---
def rate_limit(): time.sleep(0.6)

# --- Riot API helpers ---
def get_puuid(gameName, tagLine):
    key = f"{gameName}#{tagLine}"
    if key in name_cache: return name_cache[key]
    rate_limit()
    url = f"https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}"
    puuid = requests.get(url).json().get('puuid')
    name_cache[key] = puuid
    return puuid

def get_idtag_from_puuid(puuid):
    if puuid in name_cache: return {'gameName': name_cache[puuid]}
    rate_limit()
    url = f"https://{region}.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}"
    r = requests.get(url).json()
    name_cache[puuid] = r.get('gameName', 'NOT_FOUND')
    return r

def get_game_name_only(puuid):
    if puuid not in name_cache:
        rate_limit()
        try:
            r = get_idtag_from_puuid(puuid)
            name_cache[puuid] = r['gameName']
        except:
            name_cache[puuid] = 'NOT_FOUND'
    return name_cache[puuid]

def get_summoner_id(puuid):
    if puuid in summoner_id_cache: return summoner_id_cache[puuid]
    rate_limit()
    url = f"https://{platform}.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}?api_key={api_key}"
    sid = requests.get(url).json().get('id')
    summoner_id_cache[puuid] = sid
    return sid

def get_rank(puuid):
    sid = get_summoner_id(puuid)
    if sid in rank_cache: return rank_cache[sid]
    rate_limit()
    url = f"https://{platform}.api.riotgames.com/lol/league/v4/entries/by-summoner/{sid}?api_key={api_key}"
    r = requests.get(url).json()
    for entry in r:
        if entry['queueType'] == 'RANKED_SOLO_5x5':
            rank = f"{entry['tier']} {entry['rank']}"
            rank_cache[sid] = rank
            return rank
    rank_cache[sid] = "Unranked"
    return "Unranked"

def get_match_ids(puuid):
    rate_limit()
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={api_key}"
    return requests.get(url).json()

def get_match_json(match_id):
    rate_limit()
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
    return requests.get(url).json()

# --- PROCESS MATCH ---
def process_match_json(match_json, puuid):
    info = match_json['info']
    metadata = match_json['metadata']
    participants = info['participants']
    main_player = next(p for p in participants if p['puuid'] == puuid)

    row = {
        'match_id': metadata['matchId'],
        'game_mode': info['gameMode'],
        'queue_id': info['queueId'],
        'ranked': info['queueId'] == 420,
        'riot_id': main_player['riotIdGameName'],
        'riot_tag': main_player['riotIdTagline'],
        'win': main_player['win'],
        'champion': main_player['championId'],
        'champion_level': main_player['champLevel'],
        'kills': main_player['kills'],
        'deaths': main_player['deaths'],
        'assists': main_player['assists'],
        'gold_earned': main_player['goldEarned'],
        'total_damage_dealt': main_player['totalDamageDealtToChampions'],
        'vision_score': main_player['visionScore'],
        'game_creation': info['gameCreation'],
        'game_start_timestamp': info['gameStartTimestamp'],
        'game_end_timestamp': info['gameEndTimestamp'],
    }
    for i in range(7):
        row[f'item{i}'] = main_player.get(f'item{i}')

    own_team = main_player['teamId']
    allies = [p for p in participants if p['teamId'] == own_team and p['puuid'] != puuid][:4]
    enemies = [p for p in participants if p['teamId'] != own_team][:5]

    for group, players in zip(['ally', 'enemy'], [allies, enemies]):
        for i, p in enumerate(players):
            label = f'{group}_{i+1}'
            row[f'{label}'] = get_game_name_only(p['puuid'])
            row[f'{label}_champion'] = p['championId']
            row[f'{label}_position'] = p['individualPosition']
            row[f'{label}_rank'] = get_rank(p['puuid'])
            for j in range(7):
                row[f'{label}_item{j}'] = p.get(f'item{j}')
    return pd.DataFrame([row])

# --- SCAN ---
start_puuid = get_puuid(game_name, tag_line)
puuid_queue = [start_puuid]
processed_puuids, seen_matches, results = set(), set(), []

while puuid_queue and len(seen_matches) < max_total_games:
    puuid = puuid_queue.pop(0)
    if puuid in processed_puuids: continue

    print(f"🧠 Scanning: {puuid}")
    try:
        match_ids = get_match_ids(puuid)[:games_per_player]
        for match_id in match_ids:
            if match_id in seen_matches or len(seen_matches) >= max_total_games:
                continue

            seen_matches.add(match_id)
            try:
                match_json = get_match_json(match_id)

                if match_json['info'].get('gameMode') not in ['CLASSIC', 'SWIFTPLAY']:
                    continue

                df_row = process_match_json(match_json, puuid)
                results.append(df_row)

                for p in match_json['metadata']['participants']:
                    if p not in processed_puuids:
                        puuid_queue.append(p)

            except Exception as e:
                print(f"⚠️ Match failed: {match_id} → {e}")
        processed_puuids.add(puuid)
    except:
        continue

# --- FINAL CLEANUP ---
df = pd.concat(results, ignore_index=True)

# Timestamps
for col in ['game_creation', 'game_start_timestamp', 'game_end_timestamp']:
    df[col] = pd.to_datetime(df[col], unit='ms')

df['loading_screen_time'] = df['game_creation'].dt.time
df['game_start_time'] = df['game_start_timestamp'].dt.time
df['game_end_time'] = df['game_end_timestamp'].dt.time

df['game_day_of_week'] = df['game_start_timestamp'].dt.dayofweek
df['game_month'] = df['game_start_timestamp'].dt.month
df['game_year'] = df['game_start_timestamp'].dt.year

# --- APPLY DICTIONARIES TO MATCH FINAL FORMAT ---
df = df.replace(perk_dict).replace(perk_styles_dict)
df['champion'] = df['champion'].replace(champ_dict)

for i in range(7):
    col = f'item{i}'
    if col in df.columns:
        df[col] = df[col].replace(item_dict)

for role in ['ally', 'enemy']:
    for n in range(1, 6):
        for i in range(7):
            col = f'{role}_{n}_item{i}'
            if col in df.columns:
                df[col] = df[col].replace(item_dict)

for role in ['ally', 'enemy']:
    for n in range(1, 6):
        col = f'{role}_{n}_champion'
        if col in df.columns:
            df[col] = df[col].replace(champ_dict)

# --- EXPORT ---
df.to_csv("DW_data_FULL.csv", index=False)
print(f"\n✅ DONE! Saved {len(df)} rows to DW_data_FULL.csv")


🧠 Scanning: e6gR3aKcGDT6MoaI4Jett0L7kxScvYTWkjzDfhNyp3mwYzL0MsLmgBJ_wH30Twq9JLO0u7Sxx9si1w
🧠 Scanning: wkpz1qoegORFXOvxlurDGf7_OHw_GMZqTSL93EeeJ0pilSXG9el9PULJfSs07r2k-Lvs0RZz82gAGw
🧠 Scanning: KJXcrQ_oyhf6KtRie7jq92zxcsaSsgi_6OmQdTEDLvKIT8Me6g90URDh6VQrH7wNOMwsL4dAzLH4zQ
🧠 Scanning: 8q3FfAmRTWsHNHhMYdAKPyyMvaX9l5jxpbUhLUkb9YYN5BwVi1hODs6SQR14TwlFPDMBQljNhkeRSA
🧠 Scanning: mVjOBMrOIC00vNKhLiKzjRv72aeVAkcS7ewMKwZLtDOLyhM5eiWX3hBE9W1az2rQaMc2cCL7l90ukw
🧠 Scanning: iZA8gaN9-Oj0MFW3vrUYTXJe3-LT2GtN29OHEizntTzlrVJYnapTwbMCqGSxkGRAU08lwPBjU-vHVQ
🧠 Scanning: ozJaM8NaCJ9-F88QzBRRFJiVU0F39KjdjTsaYISs2HHK6se9gDbWcGv-fAqYGiBEgH_2J7gCg2kltA
🧠 Scanning: PebyINsIvvRfcLs4p_7JKMMQgTX7kXW8ICe50yYxg3BCOcKQQdl1Fdj2E5gnfIztw262bVUdbXPpug
🧠 Scanning: Id8v5zx2oXWYp6GA-w3196aA5838xVhHKIXaNA-eTGGjNFKiChvLNTaTYalw6xm4XV2oxmeYifqVmQ
🧠 Scanning: rb-ZFseLXQXuP7Kn0mN0phEqqaltIO2oSRb3JoItPlotNzhGFDijytK2Tcbh-BR3f8mphEWp9ASdCA
🧠 Scanning: KJvfISne34K9jmAi-EAz07rZIgX5COojouHGe3aeOHDUNO4Au7tVVlSJlbwljY_LdkFmeS2z4nT6zw

In [6]:
df

,match_id,game_mode,queue_id,ranked,riot_id,riot_tag,win,champion,champion_level,kills,deaths,assists,gold_earned,total_damage_dealt,vision_score,game_creation,game_start_timestamp,game_end_timestamp,item0,item1,item2,item3,item4,item5,item6,ally_1,ally_1_champion,ally_1_position,ally_1_rank,ally_1_item0,ally_1_item1,ally_1_item2,ally_1_item3,ally_1_item4,ally_1_item5,ally_1_item6,ally_2,ally_2_champion,ally_2_position,ally_2_rank,ally_2_item0,ally_2_item1,ally_2_item2,ally_2_item3,ally_2_item4,ally_2_item5,ally_2_item6,ally_3,ally_3_champion,ally_3_position,ally_3_rank,ally_3_item0,ally_3_item1,ally_3_item2,ally_3_item3,ally_3_item4,ally_3_item5,ally_3_item6,ally_4,ally_4_champion,ally_4_position,ally_4_rank,ally_4_item0,ally_4_item1,ally_4_item2,ally_4_item3,ally_4_item4,ally_4_item5,ally_4_item6,enemy_1,enemy_1_champion,enemy_1_position,enemy_1_rank,enemy_1_item0,enemy_1_item1,enemy_1_item2,enemy_1_item3,enemy_1_item4,enemy_1_item5,enemy_1_item6,enemy_2,enemy_2_champion,enemy_2_position,enemy_2_rank,enemy_2_item0,enemy_2_item1,enemy_2_item2,enemy_2_item3,enemy_2_item4,enemy_2_item5,enemy_2_item6,enemy_3,enemy_3_champion,enemy_3_position,enemy_3_rank,enemy_3_item0,enemy_3_item1,enemy_3_item2,enemy_3_item3,enemy_3_item4,enemy_3_item5,enemy_3_item6,enemy_4,enemy_4_champion,enemy_4_position,enemy_4_rank,enemy_4_item0,enemy_4_item1,enemy_4_item2,enemy_4_item3,enemy_4_item4,enemy_4_item5,enemy_4_item6,enemy_5,enemy_5_champion,enemy_5_position,enemy_5_rank,enemy_5_item0,enemy_5_item1,enemy_5_item2,enemy_5_item3,enemy_5_item4,enemy_5_item5,enemy_5_item6,loading_screen_time,game_start_time,game_end_time,game_day_of_week,game_month,game_year
0,NA1_5252668740,CLASSIC,400,False,HGILLIS,NA1,False,Darius,18,14,5,7,19129,34444,38,2025-03-23 21:36:17.088,2025-03-23 21:36:54.594,2025-03-23 22:23:13.081,Dead Man's Plate,Youmuu's Ghostblade,Stridebreaker,Force of Nature,Boots of Swiftness,Sterak's Gage,Stealth Ward,Tree,Garen,TOP,BRONZE I,Phantom Dancer,Stridebreaker,Berserker's Greaves,Serpent's Fang,Mortal Reminder,Dead Man's Plate,Farsight Alteration,StonedClone,Naafiri,MIDDLE,Unranked,Eclipse,Ionian Boots of Lucidity,Black Cleaver,Spear of Shojin,Hexdrinker,Caulfield's Warhammer,Stealth Ward,Phoenix,Anivia,BOTTOM,GOLD II,Seraph's Embrace,Sorcerer's Shoes,Malignance,Liandry's Torment,Rabadon's Deathcap,Blasting Wand,Farsight Alteration,ColmNikonias,Poppy,UTILITY,SILVER II,Bloodsong,Eclipse,Synchronized Souls,Youmuu's Ghostblade,Umbral Glaive,Fimbulwinter,Oracle Lens,dnsgfs,Gangplank,TOP,Unranked,Lord Dominik's Regards,The Collector,Trinity Force,Ionian Boots of Lucidity,Infinity Edge,Immortal Shieldbow,Stealth Ward,Kirbkek,Viego,JUNGLE,Unranked,Sundered Sky,Kraken Slayer,Armored Advance,Death's Dance,Pickaxe,Tunneler,Stealth Ward,IG0DI0FIW4RI,Cho'Gath,MIDDLE,Unranked,"Jak'Sho, The Protean",Heartsteel,Dead Man's Plate,Sunfire Aegis,Thornmail,Unending Despair,Stealth Ward,Loogi200,Twitch,BOTTOM,BRONZE I,Blade of The Ruined King,Zhonya's Hourglass,Gunmetal Greaves,Nashor's Tooth,Shadowflame,Needlessly Large Rod,Farsight Alteration,RTummis,Shen,UTILITY,Unranked,Solstice Sleigh,0,Randuin's Omen,Thornmail,Heartsteel,Mercury's Treads,Oracle Lens,21:36:17.088000,21:36:54.594000,22:23:13.081000,6,3,2025
1,NA1_5252662026,CLASSIC,400,False,HGILLIS,NA1,False,Darius,3,1,2,2,1581,972,0,2025-03-23 21:28:01.692,2025-03-23 21:28:32.096,2025-03-23 21:32:17.937,Gustwalker Hatchling,Long Sword,Long Sword,0,0,0,Stealth Ward,Tree,Garen,TOP,BRONZE I,Doran's Shield,Tiamat,Boots,0,0,0,Stealth Ward,kyòwó,Ziggs,Invalid,Unranked,Total Biscuit of Everlasting Will,0,0,0,0,0,0,Phoenix,Nilah,BOTTOM,GOLD II,Doran's Shield,Serrated Dirk,Control Ward,0,0,0,Oracle Lens,ColmNikonias,Karma,UTILITY,SILVER II,Health Potion,World Atlas,0,0,0,0,Stealth Ward,GummiNick,Volibear,TOP,GOLD II,Boots,Doran's Ring,0,0,0,0,Stealth Ward,DragonLordDrako,Kayn,JUNGLE,GOLD II,Mosstomper Seedling,0,0,0,0,0,Stealth Ward,Gigaa,LeBlanc,MIDDLE,PLATINUM IV,Dark Seal,Refillable Potion,Amplifying Tome,